In [9]:
from multiprocessing.managers import BaseManager
def connect_queue(
    address=("192.168.15.87", 8124),
    authkey=b"liujunjieabracadabra",
    queue_name="abnormal_detection_1",
    max_retries=5,
    retry_delay=5
):
    """连接到队列服务
    
    Args:
        address: 服务地址和端口
        authkey: 认证密钥
        queue_name: 队列名称
        max_retries: 最大重试次数
        retry_delay: 重试延迟(秒)
    
    Returns:
        queue: 数据队列对象
    """
    # 注册队列
    class QueueManager(BaseManager):
        pass
    
    QueueManager.register(queue_name)
    
    # 尝试连接
    retries = 0
    while retries < max_retries:
        try:
            manager = QueueManager(address=address, authkey=authkey)
            manager.connect()
            queue = getattr(manager, queue_name)()
            return queue
            
        except Exception as e:
            retries += 1
            if retries < max_retries:
                continue
    
    raise ConnectionError(f"Failed to connect after {max_retries} attempts")


def test_queue():
    """测试队列服务"""
    # 连接队列
    queue = connect_queue()
    
    # 获取并打印数据
    try:
        while True:
            try:
                # 获取一批数据
                batch = queue.get(timeout=5)
                
                # 打印批次信息
                print(f"Received batch of {len(batch)} samples")
                
                # 打印第一个样本的信息
                return batch
            except queue.Empty:
                print("Queue is empty, waiting for data...")
                continue
                
    except KeyboardInterrupt:
        print("Test stopped by user")
    except Exception as e:
        print(f"Error during testing: {e}")

In [10]:
sample = test_queue()

Received batch of 16 samples


In [11]:
sample[0]

{'utt': '0',
 'audio_data': array([-5.0054368e-06, -3.2568147e-05,  4.7808280e-06, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00], dtype=float32),
 'sr': 16000,
 'label': '0',
 'speech_token': array([1894,  162,  496,  707, 1855, 2742, 4513, 4488, 4777, 2510, 1862,
        1978, 1950, 1707, 2031, 2031, 1987, 5748, 5667, 2833, 2031,  162,
        1225,  710, 1856,  555, 4513, 4488, 4776, 2510, 2006, 1735, 2112,
        4299, 1978, 3481, 5019, 5100, 5021, 1167,   87,  162, 3412,  680,
        1370, 1122, 4513, 4488, 4749, 4697, 1762, 1950, 2031, 2031, 1987,
        5667, 5019, 5100, 2671, 1788,  165,  738,  683, 1451,  321, 4515,
        4488, 2481, 3239, 1763, 2031, 1951, 5668, 5100, 4858, 1059,  162,
         495,  707, 2090,  555, 4516, 2301, 4697, 1762, 1978, 1707, 1950,
        1950, 2014, 5667, 5100, 4858, 1896,  165, 2997,  494, 1442, 1858,
        4758, 4569, 2562, 2591, 1816, 1950, 1701, 1701, 1701, 1701, 2031,
        4860, 4860, 2673, 2018, 5015, 4754, 6068, 5987

In [2]:
import torch

model = torch.load("/datadisk2/liujunjie/cmm/audio/AutoSubAlign/autosubalign/checkpoints/SenseVoiceSmall/model.pt",map_location="cpu")

In [6]:
init_model = {}
for k, v in model.items():
    if k.startswith("encoder"):
        init_model[k] = v


In [7]:
torch.save(init_model, "init_model.pt")